In [100]:
import requests
from bs4 import BeautifulSoup
import json

In [101]:
tournament_search_URL_1 = 'https://www.mtggoldfish.com/tournament_searches/create?utf8=%E2%9C%93&tournament_search%5Bname%5D=&tournament_search%5Bformat%5D=pauper&tournament_search%5Bdate_range%5D=07%2F01%2F2020+-+03%2F31%2F2021&commit=Search'
tournament_search_URL_page = 'https://www.mtggoldfish.com/tournament_searches/create?commit=Search&page={page}&tournament_search%5Bdate_range%5D=07%2F01%2F2020+-+03%2F31%2F2021&tournament_search%5Bformat%5D=pauper&tournament_search%5Bname%5D=&utf8=%E2%9C%93'
decklist = dict()

In [102]:
def scrape_decklist(deck_id: str, decklist: dict) -> dict:
    URL = f'https://www.mtgtop8.com/event{deck_id}'
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    for a in soup.find_all(class_='L14'):
        cardname = str(a).split('>')[1].split('<')[0]
        if cardname not in decklist:
            decklist[cardname] = 0
        decklist[cardname] += 4 # TODO assumption 
        # print(f'\t\t{cardname}')
    return decklist

In [103]:
def scrape_tournament(tournament_id: str, decklist: dict) -> dict:
    URL = f'https://www.mtgtop8.com/{tournament_id}'
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    visited_decks = {}
    for a in soup.find_all('a', href=True):
        if a['href'].endswith('PAU') and a['href'].startswith('?'):
            print('href', a['href'])
            if a['href'] not in visited_decks:
                decklist = scrape_decklist(a['href'], decklist)
                visited_decks[a['href']] = True
    return decklist

In [104]:
def scrape_website(URL: str, decklist: dict) -> dict:
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, 'html.parser')
    for a in soup.find_all('a', href=True):
        if a['href'].startswith('event'):
            print(a['href'])
            decklist = scrape_tournament(a['href'], decklist)
    return decklist

In [105]:
scrape_website('https://www.mtgtop8.com/format?f=PAU', decklist)

event?e=59559&f=PAU
href ?e=59559&d=646439&f=PAU
href ?e=59559&d=646438&f=PAU
href ?e=59559&d=646438&f=PAU
href ?e=59559&d=646439&f=PAU
href ?e=59559&d=646439&f=PAU
href ?e=59559&d=646440&f=PAU
href ?e=59559&d=646440&f=PAU
href ?e=59559&d=646441&f=PAU
href ?e=59559&d=646441&f=PAU
href ?e=59559&d=646442&f=PAU
href ?e=59559&d=646442&f=PAU
href ?e=59559&d=646445&f=PAU
href ?e=59559&d=646445&f=PAU
href ?e=59559&d=646443&f=PAU
href ?e=59559&d=646443&f=PAU
href ?e=59559&d=646444&f=PAU
href ?e=59559&d=646444&f=PAU
event?e=59559&f=PAU
href ?e=59559&d=646439&f=PAU
href ?e=59559&d=646438&f=PAU
href ?e=59559&d=646438&f=PAU
href ?e=59559&d=646439&f=PAU
href ?e=59559&d=646439&f=PAU
href ?e=59559&d=646440&f=PAU
href ?e=59559&d=646440&f=PAU
href ?e=59559&d=646441&f=PAU
href ?e=59559&d=646441&f=PAU
href ?e=59559&d=646442&f=PAU
href ?e=59559&d=646442&f=PAU
href ?e=59559&d=646445&f=PAU
href ?e=59559&d=646445&f=PAU
href ?e=59559&d=646443&f=PAU
href ?e=59559&d=646443&f=PAU
href ?e=59559&d=646444&f=PAU
hre

{'Great Furnace': 344,
 'Mountain': 464,
 'Goblin Blast-Runner': 140,
 'Goblin Bushwhacker': 144,
 'Goblin Tomb Raider': 160,
 'Reckless Lackey': 40,
 'Voldaren Epicure': 216,
 'Fireblast': 72,
 'Galvanic Blast': 348,
 'Goblin Grenade': 88,
 'Kuldotha Rebirth': 144,
 'Lightning Bolt': 296,
 'Experimental Synthesizer': 168,
 'Implement of Combustion': 140,
 'Seal of Fire': 100,
 'Cast into the Fire': 320,
 'End the Festivities': 156,
 'Gorilla Shaman': 472,
 'Pyroblast': 248,
 'Relic of Progenitus': 252,
 'Contaminated Aquifer': 56,
 'Ice Tunnel': 12,
 'Island': 216,
 'Gurmag Angler': 48,
 'Sneaky Snacker': 128,
 'Tolarian Terror': 136,
 'Brainstorm': 196,
 'Cast Down': 300,
 'Counterspell': 240,
 'Deep Analysis': 128,
 'Lórien Revealed': 204,
 'Mental Note': 96,
 'Ponder': 104,
 'Snuff Out': 180,
 'Spell Pierce': 204,
 'Thought Scour': 100,
 'Unexpected Fangs': 72,
 'Annul': 200,
 'Blue Elemental Blast': 240,
 'Hydroblast': 236,
 'Murmuring Mystic': 264,
 'Recoil': 8,
 'Rotten Reunion'

In [113]:
import pandas as pd
df = pd.DataFrame([{'name': k, 'quantity': v} for k, v in decklist.items()])
df.sort_values('quantity', ascending=False).to_clipboard(index=None, sep=',')